### Задание 4 - TF-IDF

In [ ]:
!pip install pymorphy2

In [ ]:
import nltk
import math
from nltk.corpus import stopwords
import os
nltk.download('stopwords')
nltk.download('punkt')
from bs4 import BeautifulSoup
import string
import re
import pymorphy2

In [ ]:
def tokenize(content):
    tokens_ = nltk.word_tokenize(content)
    tokens_ = [i.lower() for i in tokens_]
    tokens_ = [i for i in tokens_ if ( i not in string.punctuation )]
    tokens_ = filter(None, [re.sub(r"[^a-zа-я-]+", r"", i) for i in tokens_])

    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'как', 'в', 'к',
                         'на', 'о', 'при', 'из-за', 'за', 'ао', 'но', 'х',
                         'хотя', 'среди', 'помимо', 'с'])
    tokens_ = [i for i in tokens_ if ( i not in stop_words )]

    stop_words2 = stopwords.words('english')
    stop_words2.extend(['the', 'e', 'a', 'd', 'b', 'x', 'c'])
    tokens_ = [i for i in tokens_ if ( i not in stop_words2 )]

    tokens_ = [i.replace("«", "").replace("»", "") for i in tokens_]

    return tokens_

In [10]:
def tokenize_list_content(content):
    result = []
    tokens = set() 
    for c in content:
      tokens_ = tokenize(c)
      for t_ in tokens_:
        tokens.add(t_)
      result += [tokens]
    return result

In [112]:
# словарь (номер документа -> количество слов (контент (заголовок + содержимое статьи)))
content_count_of_words = {}
content_words = {}
content_ = []
content = {}

i = 1
for file_ in os.listdir('links'):
  if (file_.__contains__('.html')):
    path_ = os.path.join('links', file_)
    file_text = open(path_).read()
    
    soup = BeautifulSoup(file_text)
    words = set()
    page_content = []

    for text in soup.find('h1', {'class': 'tm-article-snippet__title tm-article-snippet__title_h1'}):
      title = text.get_text()
      title_tokens = tokenize(title)
      for token in title_tokens:
        words.add(token)
      page_content.append(title)

    data = soup.find('div', {'xmlns': 'http://www.w3.org/1999/xhtml'})
    for p in data.find_all('p'):
       text = p.get_text()
       text_tokens = tokenize(text)
       for token in text_tokens:
          words.add(token)
       page_content.append(text)
    
    sum = 0
    for str_ in page_content:
      sum += len(str_.split())

    content_words[i] = words
    content[i] = page_content
    content_.append(page_content)
    content_count_of_words[i] = sum
    i += 1

In [7]:
lemmas_ = open('lemmas.txt').read()
lemmas_ = lemmas_.split('\n')
lemmas = {}
for l in lemmas_:
  l = l.split(':')
  lemmas[l[0]] = l[1].split(' ')

In [9]:
# создадим индекс лемм (индекс токенов был сделан в задании 3)
import json 
with open('inverted_index.txt', encoding="utf-8") as file:
    rows = [row.strip() for row in file]

index_tokens = {}
for row in rows:
    row = row.replace('\'', '\"')
    dict_ = json.loads(row)
    index_tokens[dict_["word"]] = [dict_["count"], dict_["inverted_array"]]

In [11]:
import pymorphy2

index_lemmas={}
lemmas_list = list(dict.fromkeys(lemmas))
morph = pymorphy2.MorphAnalyzer()
for i in range(1, 101):
    data = []
    for c in tokenize_list_content(content[i]):
        data += c
    data = list(dict.fromkeys(data))
    for token in data:
        lemma_from_token = morph.parse(token)[0].normal_form
        if lemma_from_token in index_lemmas.keys():
            index_lemmas[lemma_from_token].append(i)
        else:
            index_lemmas[lemma_from_token] = [i]

##### Вспомогательные функции для подсчета tf-idf


In [58]:
def get_tokens_from_doc(id):
  tokens = []
  for token in index_tokens:
    if (id in index_tokens[token][1]):
      tokens.append(token)
  return tokens

In [28]:
def get_lemmas_from_doc(id):
  lemmas = []
  for lemma in index_lemmas:
    if (id in index_lemmas[lemma]):
      lemmas.append(lemma)
  return lemmas

In [142]:
def get_lemmas(tokens): 
  lemmas = {}
  for token in tokens:
    lemma = morph.parse(token)[0].normal_form
    flag = lemmas.get(lemma)
    if flag is None:
      lemmas[lemma] = token
    else:
      if token not in flag:
        lemmas[lemma] = flag + ' ' + token
  return lemmas

In [123]:
import collections

# tf(t,d) = count of t in d / number of words in d
def find_tf_tokens(content, id):
    tokens_tfs = {}
    tokens_counter = collections.Counter(content)
    for i in tokens_counter:
        tokens_tfs[i] = tokens_counter[i] / content_count_of_words[id]
    return tokens_tfs

# idf(t) = log(N / (df + 1))
def find_idf_tokens(content_words):
  tokens_idf = {}
  for token in content_words:
    df_tokens = set()
    for id in content:
      if token in get_tokens_from_doc(id):
        df_tokens.add(id)
    tokens_idf[token] = math.log10(100 / len(df_tokens))
  return tokens_idf

In [149]:
# tf(t,d) = count of l in d / number of words in d
def find_tf_lemmas(lemmas, content, id):
  lemmas_tfs = {}
  content = ' '.join(content)
  for l in lemmas:
    count_of_l = 0
    for value in lemmas[l]:
      count_of_l += content.count(l)
    lemmas_tfs[l] = count_of_l / content_count_of_words[id]
  return lemmas_tfs

# idf(t) = log(N / (df + 1))
def find_idf_lemmas(lemmas):
  lemmas_idf = {}
  for l in lemmas:
    df_lemmas = set()
    for id in content:
      if l in get_lemmas_from_doc(id):
        df_lemmas.add(id)
    lemmas_idf[l] = math.log10(100 / len(df_lemmas))
  return lemmas_idf

In [125]:
for id in content_words:
  tokens_tf = find_tf_tokens(content_words[id], id)
  tokens_idf = find_idf_tokens(content_words[id])
  
  file_t = open('tf-idf_tokens/' + str(id) + '.txt', 'w+')
  for token in content_words[id]:
    tfidf = tokens_tf[token] * tokens_idf[token]
    file_t.write(token + ' ' + str(tokens_idf[token]) + ' ' + str(tfidf) + '\n')
  file_t.close()

In [150]:
for id in content_words:
  lemmas = get_lemmas(content_words[id])

  lemmas_tf = find_tf_lemmas(lemmas, content[id], id)
  lemmas_idf = find_idf_lemmas(lemmas)

  file_l = open('tf-idf_lemmas/' + str(id) + '.txt', 'w+')
  for lemma in lemmas:
    tfidf = lemmas_tf[lemma] * lemmas_idf[lemma]
    file_l.write(lemma + ' ' + str(lemmas_idf[lemma]) + ' ' + str(tfidf) + '\n')
  file_l.close()

In [ ]:
!zip -r tf-idf_tokens.zip tf-idf_tokens

In [ ]:
!zip -r tf-idf_lemmas.zip tf-idf_lemmas

In [127]:
from google.colab import files

In [128]:
files.download("tf-idf_tokens.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [153]:
files.download("tf-idf_lemmas.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>